In [1]:
import numpy as np
from preprocessing.text import tokenize
from models.llm import load_llm
from models.classifier import load_classifier
from tqdm import tqdm
from preprocessing.image import preprocess_and_check_image
from database.connect import conn
from preprocessing.document import extract_text_from_pdf
from tqdm import tqdm

/home/nightstalker/miniconda3/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = load_llm()
trainer, tokenizer = load_classifier()

In [3]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM Baby_Bath_Skin_and_Grooming") 
rows = cursor.fetchall()

In [4]:
IMAGE_INDEX = 4
DOC_INDEX = None
PRIMARY_KEY = ['id']
user_prompt = "Does it have a baby picture on it that is seen easily in the package?"
picture_prompt = "analyze the picture"
row_prompt = "analyze the row data"
document_prompt = "analyze the document"

image_flag = 1
document_flag = 0

columns = [col[0] for col in cursor.description]
pk_index = [i for i in range(len(columns)) if columns[i] in PRIMARY_KEY]

columns, pk_index

(['id',
  'name',
  'main_category',
  'sub_category',
  'image',
  'ratings',
  'no_of_ratings',
  'discount_price',
  'actual_price'],
 [0])

In [6]:
results_text = []
PK = []
indices = []
predicted_classes = []

def text_llm(row):
    prompt = f"{row_prompt}.\n{user_prompt}"
    text_data = " | ".join([f"{columns[j]}: {row[j]}" for j in range(len(columns)) if j != IMAGE_INDEX])
    result = llm.generate_content(
        [f"Row data: {text_data}\n\n", prompt]
    )
    return result.text

def image_llm(image, row):
    prompt = f"{picture_prompt}, and {row_prompt}.\n{user_prompt}"
    text_data = " | ".join([f"{columns[j]}: {row[j]}" for j in range(len(columns)) if j != IMAGE_INDEX])
    result = llm.generate_content(
        [image, "\n\n", f"Row data: {text_data}\n\n", prompt]
    )
    return result.text

def document_llm(document, row):
    prompt = f"{document_prompt}, and {row_prompt}.\n{user_prompt}"
    text_data = " | ".join([f"{columns[j]}: {row[j]}" for j in range(len(columns)) if j != DOC_INDEX])
    result = llm.generate_content(
        [document, "\n\n", f"Row data: {text_data}\n\n", prompt]
    )
    return result.text

def image_document_llm(image, document, row):
    prompt = f"{picture_prompt}, {document_prompt}, and {row_prompt}.\n{user_prompt}"
    text_data = " | ".join([f"{columns[j]}: {row[j]}" for j in range(len(columns)) if j != IMAGE_INDEX and j != DOC_INDEX])
    result = llm.generate_content(
        [image, document, "\n\n", f"Row data: {text_data}\n\n", prompt]
    )
    return result.text


for i in tqdm(range(len(rows[:20]))):

    if not image_flag and not document_flag:
        result_text = text_llm(rows[i])
    
    elif image_flag and not document_flag:
        image = preprocess_and_check_image(rows[i][IMAGE_INDEX])
        if image:
            result_text = image_llm(image, rows[i])
        else:
            result_text = text_llm(rows[i])

    elif not image_flag and document_flag:
        ## Apply logic of reading pdf
        document = extract_text_from_pdf(rows[i][DOC_INDEX])
        if document:
            result_text = document_llm(document, rows[i])
        else:
            result_text = text_llm(rows[i])
        
    elif image_flag and document_flag:
        document = extract_text_from_pdf(rows[i][DOC_INDEX])
        image = preprocess_and_check_image(rows[i][IMAGE_INDEX])
        if document and image:
            result_text = image_document_llm(image, document, rows[i])
        elif document and image:
            result_text = document_llm(document, rows[i])
        elif not document and image:
            result_text = image_llm(image, rows[i])
        elif not document and not image:
            result_text = text_llm(rows[i])

        
    classifier_text = f"Question: {user_prompt} Answer: {result_text}"
    results_text.append(result_text)
    dataset_subset = tokenize(classifier_text, tokenizer)
    predictions = trainer.predict(dataset_subset)

    predicted_class = np.argmax(predictions.predictions, axis=1)[0]
    predicted_classes.append(predicted_class)
    
    if predicted_class <= 1:
        indices.append(i)
        PK.append([rows[i][j] for j in pk_index])

  0%|                                                                                                                                                                                                                  | 0/20 [00:00<?, ?it/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mehdinejjar (mehdinejjar-al-akhawayn-university). Use `wandb login --relogin` to force relogin


  5%|██████████                                                                                                                                                                                                | 1/20 [00:04<01:18,  4.11s/it]

 10%|████████████████████▏                                                                                                                                                                                     | 2/20 [00:06<00:57,  3.22s/it]

 15%|██████████████████████████████▎                                                                                                                                                                           | 3/20 [00:09<00:48,  2.83s/it]

...

 80%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                        | 16/20 [00:45<00:10,  2.68s/it]

 85%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                              | 17/20 [00:47<00:07,  2.55s/it]

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                    | 18/20 [00:50<00:05,  2.58s/it]

 95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉          | 19/20 [00:52<00:02,  2.48s/it]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:55<00:00,  2.77s/it]


In [7]:
results_text, PK, indices, predicted_classes

(["Yes, the product clearly features a picture of a baby playing with a stacking toy.  It's prominent on the front label and easily noticeable. This aligns with the product description indicating its use for cleaning baby items.\n",
  'No, the product image shows a baby bather with illustrations of sea creatures (narwhal, shark, octopus, fish, turtle), not actual photographs or images of babies. While designed for babies, the imagery used is cartoonish and focused on an underwater theme.\n',
  "Yes, the product packaging clearly displays a stylized illustration of a baby's head, emphasizing that the product is for babies. This is consistent with the product name and description provided in the row data, which confirms it is a baby massage oil.\n",
  'No, the picture on the bottle is of lotus and meadowsweet flowers, not a baby. While the product is clearly marketed for babies (as indicated by "Baby Moments" and "0+"), there are no actual images of babies on the packaging.\n',
  "The pr